
### Problema 1: **Selección de Portafolio**

Un consultor en finanzas requiere elegir para uno de sus clientes un conjunto de acciones en las cuales invertir. El cliente desea invertir € 100.000 en 6 diferentes acciones. El consultor estima el retorno sobre la inversión (ROI) que el cliente puede obtener en un período de seis meses. La tabla nro. 1 muestra para cada acción, su país de origen, la categoría (T: tecnológicas, N: no tecnológicas) y el ROI esperado.

#### Variables de Decision:

$x_1$: Acciones de origen Japon de categoria T  
$x_2$: Acciones de orgen Reino Unido de categoria T  
$x_3$: Acciones de origen Francia de categoria T  
$x_4$: Acciones de origen USA de categoria N  
$x_5$: Acciones de origen Alemania de categoria N  
$x_6$: Acciones de origen Francia de categoria N  

#### Funcion Objetvo:

Maximizar ROI:  

$\frac{5.3}{100} x_1 + \frac{6.2}{100} x_2 + \frac{5.1}{100} x_3 + \frac{4.9}{100} x_4 + \frac{6.5}{100} x_5 + \frac{3.4}{100} x_6$

#### Restricciones

$x_1 + x_2 + x_3 + x_4 + x_5 + x_6$ = 100000  

$ 5000 \leq x_1 \leq 40000 $  
$ 5000 \leq x_2 \leq 40000 $  
$ 5000 \leq x_3 \leq 40000 $  
$ 5000 \leq x_4 \leq 40000 $  
$ 5000 \leq x_5 \leq 40000 $  
$ 5000 \leq x_6 \leq 40000 $  

$ x_1 + x_2 + x_3 \leq 30000$

$ x_2 + x_3 + x_5 + x_6 \geq 50000$


In [5]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modelo = Model(GLPK.Optimizer) # Instancia un modelo.
 
@variable(modelo, 5000 <= x1 <= 40000)
@variable(modelo, 5000 <= x2 <= 40000)
@variable(modelo, 5000 <= x3 <= 40000)
@variable(modelo, 5000 <= x4 <= 40000)
@variable(modelo, 5000 <= x5 <= 40000)
@variable(modelo, 5000 <= x6 <= 40000)
 
@objective(modelo, Max, (5.3/100)*x1 + (6.2/100)*x2 + (5.1/100)*x3 + (4.9/100)*x4 + (6.5/100)*x5 + (3.4/100)*x6)

@constraint(modelo, x1 + x2 + x3 + x4 + x5 + x6 <= 100000)

@constraint(modelo, 5000 <= x1 <= 40000)
@constraint(modelo, 5000 <= x2 <= 40000)
@constraint(modelo, 5000 <= x3 <= 40000)
@constraint(modelo, 5000 <= x4 <= 40000)
@constraint(modelo, 5000 <= x5 <= 40000)
@constraint(modelo, 5000 <= x6 <= 40000)

@constraint(modelo, x1 + x2 + x3 <= 30000)
@constraint(modelo, x2 + x3 + x5 + x6 >= 50000)
 
optimize!(modelo) # Resuelve el  modelo

println("Acccion x1: ", value(x1))
println("Acccion x2: ", value(x2))
println("Acccion x3: ", value(x3))
println("Acccion x4: ", value(x4))
println("Acccion x5: ", value(x5))
println("Acccion x6: ", value(x6))

valor = objective_value(modelo) # Entrega el valor objetivo óptimo

println("Valor Objetivo Optimo: ", valor)


Acccion x1: 5000.0
Acccion x2: 20000.0
Acccion x3: 5000.0
Acccion x4: 25000.0
Acccion x5: 40000.0
Acccion x6: 5000.0
Valor Objetivo Optimo: 5755.0


#### Mejora del codigo problema 1

In [2]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modelo = Model(GLPK.Optimizer) # Instancia un modelo.

rois = [5.3, 6.2, 5.1, 4.9, 6.5, 3.4]

@variable(modelo, 5000 <= x[1:6] <= 40000) 

@objective(modelo, Max, sum(rois[i]/100 * x[i] for i in 1:6))

@constraint(modelo, sum(x[i] for i in 1:3) <= 30000)

@constraint(modelo, sum(x[i] for i in [2,3,5,6]) >= 50000)

@constraint(modelo, sum(x[i] for i in 1:6) <= 100000)

optimize!(modelo)

# Imprimir resultados
for i in 1:6
    println("Accion x$i: ", value(x[i]))
end

println("Valor Objetivo Optimo: ", objective_value(modelo))


Accion x1: 5000.0
Accion x2: 20000.0
Accion x3: 5000.0
Accion x4: 25000.0
Accion x5: 40000.0
Accion x6: 5000.0
Valor Objetivo Optimo: 5755.0


### Problema 2: **Problema de planificación**

Se está prospectado una mina de uranio a cielo abierto. Sobre la base de los resultados de
algunas perforaciones de prueba, la mina se ha subdividido en unidades de explotación
denominadas bloques. Las excavaciones se deben realizar en forma aterrazada, con el fin de
permitir la circulación de los camiones. El yacimiento se extiende de este a oeste, viéndose
limitada por un pueblo y por un conjunto montañoso. Considerando estas limitaciones, se han
identificado 18 bloques de 10.000 toneladas distribuidos en tres niveles. Para extraer un bloque,
es necesario antes extraer tres bloques del nivel inmediatamente superior, siendo estos, el bloque
superior y los ubicados a la derecha y a la izquierda de este. Ejemplo: Para extraer el bloque 16,
antes se deben extraer los bloques 10, 11 y 12. 

#### Variables de Decision:

Uno para cada bloque: 

$  x_i \in \{0, 1\}, \quad i = 1, \ldots, 18  $  
0: No se extrae el bloque i  
1: Se extrae el bloque i

$  valor_i \quad i = 1, \ldots, 18  $  (Valor de cada Bloque)

$ costo_i \quad i = 1, \ldots, 18 $  (Costo de cada Bloque)

#### Funcion Objetvo:

Maximizar Utilidad: 

$ \sum_{i=1}^{18} (valor_i - costo_i) \cdot x_i $

#### Restricciones

Restricciones de no negatividad para las variables binarias

$ x i​≥0  \quad ∀i ∈{1,2,...,18} $


Restricciones de relación entre variables
     
$ x_p - x_i \geq 0 \quad \text{para } i = 9, 10, 11, 12, 13, 14 \text{ y } p = i, i+1, i+2 $

$ x_p - x_i \geq 0 \quad \text{para } i = 15, 16, 17, 18 \text{ y } p = i, i+1, i+2 $



In [1]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modelo = Model(GLPK.Optimizer) # Instancia un modelo.

toneladas_por_bloque = 10000

#Valores mercado de cada bloque ordenados
valor_asignados = [200 0 0 0 0 0 300 0 0 500 0 200 0 0 0 0 1000 1200]
valor_asignados = valor_asignados * toneladas_por_bloque

#Costos de cada bloque ordenados 
costos_asignados = [100 100 100 100 100 100 100 100 1000 200 200 200 200 1000 1000 1000 300 1000]
costos_asignados = costos_asignados * toneladas_por_bloque
 
@variable(modelo, x[1:18] , Bin) #Binario 
  
@objective(modelo, Max, sum((valor_asignados[i] - costos_asignados[i]) * x[i] for i in 1:18))

@constraint(modelo, x[1] - x[9] >= 0)
@constraint(modelo, x[2] - x[9] >= 0)
@constraint(modelo, x[3] - x[9] >= 0)   

@constraint(modelo, x[2] - x[10] >= 0)
@constraint(modelo, x[3] - x[10] >= 0)
@constraint(modelo, x[4] - x[10] >= 0) 

@constraint(modelo, x[3] - x[11] >= 0)
@constraint(modelo, x[4] - x[11] >= 0)
@constraint(modelo, x[5] - x[11] >= 0) 

@constraint(modelo, x[4] - x[12] >= 0)
@constraint(modelo, x[5] - x[12] >= 0)
@constraint(modelo, x[6] - x[12] >= 0)

@constraint(modelo, x[5] - x[13] >= 0)
@constraint(modelo, x[6] - x[13] >= 0)
@constraint(modelo, x[7] - x[13] >= 0)

@constraint(modelo, x[6] - x[14] >= 0)
@constraint(modelo, x[7] - x[14] >= 0)
@constraint(modelo, x[8] - x[14] >= 0)

@constraint(modelo, x[9] - x[15] >= 0)
@constraint(modelo, x[10] - x[15] >= 0)
@constraint(modelo, x[11] - x[15] >= 0)

@constraint(modelo, x[10] - x[16] >= 0)
@constraint(modelo, x[11] - x[16] >= 0)
@constraint(modelo, x[12] - x[16] >= 0)

@constraint(modelo, x[11] - x[17] >= 0)
@constraint(modelo, x[12] - x[17] >= 0)
@constraint(modelo, x[13] - x[17] >= 0)

@constraint(modelo, x[12] - x[18] >= 0)
@constraint(modelo, x[13] - x[18] >= 0)
@constraint(modelo, x[14] - x[18] >= 0)

optimize!(modelo)

println("Valor Objetivo Optimo: ", objective_value(modelo))

# Imprimir resultados
for i in 1:18
    println("BLOQUE $i: ", value(x[i]))
end



Valor Objetivo Optimo: 4.0e6
BLOQUE 1: 1.0
BLOQUE 2: 1.0
BLOQUE 3: 1.0
BLOQUE 4: 1.0
BLOQUE 5: 1.0
BLOQUE 6: 1.0
BLOQUE 7: 1.0
BLOQUE 8: 0.0
BLOQUE 9: 0.0
BLOQUE 10: 1.0
BLOQUE 11: 1.0
BLOQUE 12: 1.0
BLOQUE 13: 1.0
BLOQUE 14: 0.0
BLOQUE 15: 0.0
BLOQUE 16: 0.0
BLOQUE 17: 1.0
BLOQUE 18: 0.0


#### Mejorar el codigo problema 2

In [2]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modelo = Model(GLPK.Optimizer) # Instancia un modelo.

toneladas_por_bloque = 10000

#valores mercado de cada bloque ordenados
valor_asignados = [200 0 0 0 0 0 300 0 0 500 0 200 0 0 0 0 1000 1200] .* toneladas_por_bloque 
 
#costos de cada bloque ordenados 
costos_asignados = [100 100 100 100 100 100 100 100 1000 200 200 200 200 1000 1000 1000 300 1000] .* toneladas_por_bloque
 
@variable(modelo, x[1:18] , Bin) 
  
@objective(modelo, Max, sum((valor_asignados[i] - costos_asignados[i]) * x[i] for i in 1:18))
 
n = 1
for i in 9:14 
    s = n + 2  
    for p in n:s
        n = n + 1 
        @constraint(modelo, x[p] - x[i] >= 0)
    end 
    n = n - 2 
end

n = 9
for i in 15:18 
    s = n + 2   
    for p in n:s
        n = n + 1
        @constraint(modelo, x[p] - x[i] >= 0) 
    end 
    n = n - 2 
end
 
optimize!(modelo)

println("Valor Objetivo Optimo: ", objective_value(modelo))

# Imprimir resultados
for i in 1:18
    println("BLOQUE $i: ", value(x[i]))
end


Valor Objetivo Optimo: 4.0e6
BLOQUE 1: 1.0
BLOQUE 2: 1.0
BLOQUE 3: 1.0
BLOQUE 4: 1.0
BLOQUE 5: 1.0
BLOQUE 6: 1.0
BLOQUE 7: 1.0
BLOQUE 8: 0.0
BLOQUE 9: 0.0
BLOQUE 10: 1.0
BLOQUE 11: 1.0
BLOQUE 12: 1.0
BLOQUE 13: 1.0
BLOQUE 14: 0.0
BLOQUE 15: 0.0
BLOQUE 16: 0.0
BLOQUE 17: 1.0
BLOQUE 18: 0.0


### Problema 3: **Problema de programación**

Un departamento de informática de una pequeña universidad se encuentra programando
los exámenes finales que deberán ser rendidos por diferentes grupos de estudiantes. La principal
condición en la confección del calendario es que todos los grupos de alumnos puedan rendir sus
exámenes sin traslapes de horario. Los siete exámenes que se deben programar se han etiquetado
desde ID1 hasta ID7. La tabla nro. 4 relaciona los exámenes que presentan a lo menos un alumno
en común en sus nóminas. ¿Cuál es el calendario de exámenes que minimiza el número de
períodos de tiempo utilizados?


- $n$ el número de exámenes

#### Variables de Decision: 

- $x_{i,j}$ una variable binaria que toma el valor 1 si el examen $i$ se asigna al período $j$, y 0 en caso contrario


#### Funcion Objetvo:
Minimizar 

$$\min \sum_{j=1}^n j \cdot \sum_{i=1}^n x_{i,j}$$


#### Restricciones:

1. Cada examen debe ser asignado a exactamente un período:
$$\sum_{j=1}^n x_{i,j} = 1, \quad \forall i \in \{1, 2, \ldots, n\}$$

2. Si dos exámenes $i$ y $j$ tienen al menos un estudiante en común, entonces no pueden ser asignados al mismo período:
$$x_{i,k} + x_{j,k} \leq 1, \quad \forall (i,j) \in E, \quad \forall k \in \{1, 2, \ldots, n\}$$
donde $E$ es el conjunto de pares de exámenes que tienen al menos un estudiante en común. (Conflictos)



Donde:
- La función objetivo busca minimizar el número de períodos utilizados, que se expresa como la suma de $x_{i,j}$
- La primera restricción asegura que cada examen $i$ se asigne a exactamente un período $j$.
- La segunda restricción garantiza que los exámenes que tienen al menos un estudiante en común no se asignen al mismo período.
  La última restricción establece que las variables $x_{i,j}$ son binarias, es decir, pueden tomar valores 0 o 1.
  

In [38]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modelo = Model(GLPK.Optimizer) # Instancia un modelo.

n = 7

#Relacion de examenes que presentan al menos un alumno en comun
conflictos = [(1,2), (1,3), (1,4), (1,7), (2,3), (2,4), (2,5), (2,7), (3,4), (3,6), (3,7), (4,5), (4,6), (5,6), (5,7), (6,7)]

@variable(modelo, x[1:n, 1:n], Bin)  # Variable de decisión

# Función objetivo
@objective(modelo, Min, sum(j * sum(x[i,j] for i in 1:n) for j in 1:n))


# Restricciones
for i in 1:n
    @constraint(modelo, sum(x[i,j] for j in 1:n) == 1)  # Cada examen debe asignarse exactamente a un período
end


for (i, j) in conflictos
    for k in 1:n
        @constraint(modelo, x[i,k] + x[j,k] <= 1)  # No asignar exámenes conflictivos al mismo período
    end
end
    
# Resolver el modelo
optimize!(modelo)


println("Valor Objetivo Optimo: ", objective_value(modelo))

for i in 1:7
    println("PERODO $i: ", value(x[i]))
end


Valor Objetivo Optimo: 16.0
PERODO 1: 0.0
PERODO 2: 1.0
PERODO 3: 0.0
PERODO 4: 0.0
PERODO 5: 0.0
PERODO 6: 1.0
PERODO 7: 0.0
